predicting the car's selling price using linear and lasso regression (regularization)

import the dependencies

In [44]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn import metrics

import requests
import csv

In [27]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


data collection

In [28]:
car_dataset = pd.read_csv("/content/drive/MyDrive/self_projects/car_price/cardekho.csv")
car_dataset.head()

name  year  selling_price  km_driven    fuel  \
0        Maruti Swift Dzire VDI  2014         450000     145500  Diesel   
1  Skoda Rapid 1.5 TDI Ambition  2014         370000     120000  Diesel   
2      Honda City 2017-2020 EXi  2006         158000     140000  Petrol   
3     Hyundai i20 Sportz Diesel  2010         225000     127000  Diesel   
4        Maruti Swift VXI BSIII  2007         130000     120000  Petrol   

  seller_type transmission         owner  mileage(km/ltr/kg)  engine  \
0  Individual       Manual   First Owner               23.40  1248.0   
1  Individual       Manual  Second Owner               21.14  1498.0   
2  Individual       Manual   Third Owner               17.70  1497.0   
3  Individual       Manual   First Owner               23.00  1396.0   
4  Individual       Manual   First Owner               16.10  1298.0   

  max_power  seats  
0        74    5.0  
1    103.52    5.0  
2        78    5.0  
3        90    5.0  
4      88.2    5.0

data preprocessing and visualization

In [31]:
car_dataset.shape

(8128, 12)

In [32]:
car_dataset.describe()

year  selling_price     km_driven  mileage(km/ltr/kg)  \
count  8128.000000   8.128000e+03  8.128000e+03         7907.000000   
mean   2013.804011   6.382718e+05  6.981951e+04           19.418783   
std       4.044249   8.062534e+05  5.655055e+04            4.037145   
min    1983.000000   2.999900e+04  1.000000e+00            0.000000   
25%    2011.000000   2.549990e+05  3.500000e+04           16.780000   
50%    2015.000000   4.500000e+05  6.000000e+04           19.300000   
75%    2017.000000   6.750000e+05  9.800000e+04           22.320000   
max    2020.000000   1.000000e+07  2.360457e+06           42.000000   

            engine        seats  
count  7907.000000  7907.000000  
mean   1458.625016     5.416719  
std     503.916303     0.959588  
min     624.000000     2.000000  
25%    1197.000000     5.000000  
50%    1248.000000     5.000000  
75%    1582.000000     5.000000  
max    3604.000000    14.000000

In [33]:
car_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                8128 non-null   object 
 1   year                8128 non-null   int64  
 2   selling_price       8128 non-null   int64  
 3   km_driven           8128 non-null   int64  
 4   fuel                8128 non-null   object 
 5   seller_type         8128 non-null   object 
 6   transmission        8128 non-null   object 
 7   owner               8128 non-null   object 
 8   mileage(km/ltr/kg)  7907 non-null   float64
 9   engine              7907 non-null   float64
 10  max_power           7913 non-null   object 
 11  seats               7907 non-null   float64
dtypes: float64(3), int64(3), object(6)
memory usage: 762.1+ KB


In [34]:
car_dataset.isnull().sum()

name                    0
year                    0
selling_price           0
km_driven               0
fuel                    0
seller_type             0
transmission            0
owner                   0
mileage(km/ltr/kg)    221
engine                221
max_power             215
seats                 221
dtype: int64

In [39]:

car_dataset["max_power"] = pd.to_numeric(car_dataset["max_power"], errors='coerce') # converting object type to numeric type
car_dataset["max_power"] = car_dataset["max_power"].fillna(car_dataset["max_power"].mean()) # filling the null values

In [42]:
car_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                8128 non-null   object 
 1   year                8128 non-null   int64  
 2   selling_price       8128 non-null   int64  
 3   km_driven           8128 non-null   int64  
 4   fuel                8128 non-null   object 
 5   seller_type         8128 non-null   object 
 6   transmission        8128 non-null   object 
 7   owner               8128 non-null   object 
 8   mileage(km/ltr/kg)  8128 non-null   float64
 9   engine              8128 non-null   float64
 10  max_power           8128 non-null   float64
 11  seats               8128 non-null   float64
dtypes: float64(4), int64(3), object(5)
memory usage: 762.1+ KB


In [41]:
# filling the null values in mileage
car_dataset["mileage(km/ltr/kg)"] = car_dataset["mileage(km/ltr/kg)"].fillna(car_dataset["mileage(km/ltr/kg)"].mean())

# filling the null values in engine
car_dataset["engine"] = car_dataset["engine"].fillna(car_dataset["engine"].mean())

# filling the null values in seats
car_dataset["seats"] = car_dataset["seats"].fillna(car_dataset["seats"].mean())

In [47]:
# identifying the unique values
print(car_dataset["fuel"].unique())
print(car_dataset["seller_type"].unique())
print(car_dataset["transmission"].unique())

[1 3 2 0]
[1 0 2]
['Manual' 'Automatic']


In [49]:
# encoding the categorical data
encoder = LabelEncoder()
car_dataset["fuel"] = encoder.fit_transform(car_dataset["fuel"])
car_dataset["seller_type"] = encoder.fit_transform(car_dataset["seller_type"])
car_dataset["transmission"] = encoder.fit_transform(car_dataset["transmission"]) # manual-1, auto-0

In [50]:
car_dataset.head()

name  year  selling_price  km_driven  fuel  \
0        Maruti Swift Dzire VDI  2014         450000     145500     1   
1  Skoda Rapid 1.5 TDI Ambition  2014         370000     120000     1   
2      Honda City 2017-2020 EXi  2006         158000     140000     3   
3     Hyundai i20 Sportz Diesel  2010         225000     127000     1   
4        Maruti Swift VXI BSIII  2007         130000     120000     3   

   seller_type  transmission         owner  mileage(km/ltr/kg)  engine  \
0            1             1   First Owner               23.40  1248.0   
1            1             1  Second Owner               21.14  1498.0   
2            1             1   Third Owner               17.70  1497.0   
3            1             1   First Owner               23.00  1396.0   
4            1             1   First Owner               16.10  1298.0   

   max_power  seats  
0      74.00    5.0  
1     103.52    5.0  
2      78.00    5.0  
3      90.00    5.0  
4      88.20    5.0

splitting the data

In [51]:
X = car_dataset.drop(["name", "year", "owner"], axis = 1)
y = car_dataset["selling_price"]

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

model training

In [53]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [54]:
# for training data
training_data_pred = lr.predict(X_train)

# for testing data
testing_data_pred = lr.predict(X_test)


In [55]:
print("r_2 score error for training data: ", metrics.r2_score(y_train, training_data_pred))
print("r_2 score error for testing data: ", metrics.r2_score(y_test, testing_data_pred))

r_2 score error for training data:  1.0
r_2 score error for testing data:  1.0


In [56]:
lasso = Lasso()
lasso.fit(X_train, y_train)

Lasso()

In [57]:
# for training data
training_data_pred = lr.predict(X_train)

# for testing data
testing_data_pred = lr.predict(X_test)


In [58]:
print("r_2 score error for training data: ", metrics.r2_score(y_train, training_data_pred))
print("r_2 score error for testing data: ", metrics.r2_score(y_test, testing_data_pred))

r_2 score error for training data:  1.0
r_2 score error for testing data:  1.0
